This is a starter notebook for the project, you'll have to import the libraries you'll need, you can find a list of the ones available in this workspace in the requirements.txt file in this workspace. 

# Step 0: Load the Vocareum Key and Initialize the Model

In [ ]:
from src.vocareum import load_vocareum_key


load_vocareum_key()

# Step 1: Setting Up the Python Application and Load the Model

In [ ]:
from src.model import Model, ImageGenerator
from src.schemas import PropertyDetails, print_schemas, CreateRealEstateListingsPrompt
from src.read_write_ops import save_df_in_csv, read_df_from_csv

from pathlib import Path

In [ ]:
llm = Model().llm

# Step 2: Generating Real Estate Listings

In [ ]:
property_example = PropertyDetails(
    neighborhood="Green Oaks",
    price=800000,
    bedrooms=3,
    bathrooms=2,
    house_size=2000,
    description="Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.",
    neighborhood_description="Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze."
)
property_example_text = print_schemas(property_example)

In [ ]:
real_state_csv_path = Path("real_estate_listings.csv")

In [ ]:
def add_generated_images_to_listings(real_state_df):
    for index, row in real_state_df.iterrows():
        try:
            output_path = f"property_{index}.png"
            image_gen = ImageGenerator(row["description"], output_path)
            real_state_df.loc[index, "image_path"] = image_gen.img_path
            image_gen.generate()
        except Exception as e:
            ValueError(f"Failed to generate image for row {index}: {e}")
    return real_state_df

In [ ]:
if real_state_csv_path.exists():
    df = read_df_from_csv(real_state_csv_path)
else:
    real_state_listings_prompt = CreateRealEstateListingsPrompt()
    query = real_state_listings_prompt.prepare_query(property_example_text=property_example_text)
    response = llm.invoke(query)
    df = real_state_listings_prompt.convert_response_to_df(response)
    df = add_generated_images_to_listings(df)
    save_df_in_csv(df, real_state_csv_path)

In [ ]:
df.head(3)

# Step 3: Storing Listings in a Vector Database

In [ ]:
from src.vector_db import RAGWithChromaClipEmbeddings

In [ ]:
chroma_db = RAGWithChromaClipEmbeddings(df)

In [ ]:
result = chroma_db.query_db('A quiet neighborhood, good local schools, and convenient shopping options.', 1)

In [ ]:
# result

In [ ]:
chroma_db.display_search_results(result)

# Step 4: Building the User Preference Interface

This step collects buyer preferences for a property using an AI-driven interactive assistant. The assistant asks predefined questions about the buyer's requirements. The AI listens to responses, stores them in memory, and generates a concise summary of the buyer's needs.

In [ ]:
from src.real_state_agent import BuyerCollectorPreferences

In [ ]:
collector = BuyerCollectorPreferences(llm=llm)

In [ ]:
collector.collect()

In [ ]:
collector.summary

Using the summarized information, we can now query the database containing our property details.

In [ ]:
result = chroma_db.query_db(collector.summary, 2)
chroma_db.display_search_results(result)

# Step 5: Searching Based on Preferences